In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  

api_key = os.getenv("MISTRAL_API_KEY")

In [3]:
messages = [
    {"role":"system","content":"You are a concise, helpful assistant."},
    {"role":"user","content":"Just say 'hello."}
]

In [4]:
import os
from mistralai import Mistral

client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])
resp = client.chat.complete(
    model="mistral-large-latest",  # or "mistral-small-latest" / "open-mistral-7b"
    messages=[
        {"role":"system","content":"You are concise and factual."},
        {"role":"user","content":"What is ReAct prompting in agents?"}
    ],
    temperature=0.7,
    max_tokens=256,
)

In [5]:
print(resp.choices[0].message.content)

**ReAct (Reasoning and Acting)** is a prompting technique for **LLM-based agents** that interleaves **reasoning traces** (thought processes) with **actions** (e.g., tool/API calls) in a structured, iterative loop. Developed by *Yao et al. (2022)*, it aims to improve transparency, interpretability, and task-solving capability by:

1. **Reasoning**: The agent generates internal thoughts (e.g., *"I need to find X to answer Y"*).
2. **Acting**: The agent takes concrete steps (e.g., searching a database, calling an API).
3. **Observation**: The agent receives feedback (e.g., API response, error message).
4. **Repeat**: The cycle continues until the task is resolved.

---
### **Key Features**
- **Interleaved Structure**:
  ```plaintext
  Thought: [reasoning step]
  Action: [tool/API call, e.g., `search[query]`]
  Observation: [result]
  ```
- **Dynamic Adaptation**: Adjusts actions based on observations (e.g., retries after failures).
- **Tool Integration**: Works with external tools (e.g., 

In [ ]:
import sys
from mistralai import Mistral

with client.chat.stream(
    model="mistral-small-latest",
    messages=[{"role": "user", "content": "Give a 3-bullet summary of LangChain."}],
    temperature=0.3,
) as stream:
    full = []
    for event in stream:
        # token chunks arrive as 'ChatCompletionChunk' objects
        if hasattr(event, "choices"):
            for choice in event.choices:
                delta = choice.delta
                if delta and delta.content:
                    sys.stdout.write(delta.content)
                    sys.stdout.flush()

In [23]:
import os
texts = [
    "RAG uses retrieval to ground generation.",
    "AutoGPT chains tool-using steps for goals."
]
emb = client.embeddings.create(model="mistral-embed", inputs=texts)
print(len(emb.data[0].embedding), "dims") 

1024 dims


In [ ]:
import os
import json
import time
from mistralai import Mistral

client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string"}
                },
                "required": ["city"]
            }
        }
    }
]
def get_weather(city: str):
    return {"city": city, "temp_c": 26, "status": "clear"}

tool_impl = {"get_weather": get_weather}

messages = [
    {"role": "user", "content": "What is the weather in Dehradun?"}
]
# 1) FIRST MODEL CALL

resp = client.chat.complete(
    model="mistral-small-latest",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
# check if tool call exists
choice = resp.choices[0]
tool_calls = choice.message.tool_calls

if tool_calls:
    call = tool_calls[0]            
    name = call.function.name
    args = json.loads(call.function.arguments)

    result = tool_impl[name](**args)

    # add tool call + result back into conversation
    messages.append({
        "role": "assistant",
        "tool_calls": [
            {
                "id": call.id,
                "function": {
                    "name": name,
                    "arguments": call.function.arguments
                },
                "type": "function"
            }
        ]
    })

    messages.append({
        "role": "tool",
        "content": json.dumps(result),
        "tool_call_id": call.id
    })

    # 2) SECOND MODEL CALL

    final = client.chat.complete(
        model="mistral-small-latest",
        messages=messages
    )
    print(final.choices[0].message.content)
else:
    print(choice.message.content)


The current weather in Dehradun is clear with a temperature of **26°C**.


In [25]:
import os, sys, json, time
from mistralai import Mistral

client = Mistral(api_key=os.environ["MISTRAL_API_KEY"])

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "now",
            "description": "Return current unix time.",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    }
]

def now():
    return {"ts": time.time()}

IMPL = {"now": now}

def run():
    msgs = [{"role": "system", "content": "Answer briefly. Use tools if needed."}]

    while True:
        user = input("> ").strip()
        if not user or user.lower() in {"exit", "quit"}:
            break

        msgs.append({"role": "user", "content": user})

        # First call: see if model wants to use a tool
        resp = client.chat.complete(
            model="mistral-small-latest",
            messages=msgs,
            tools=TOOLS,
            tool_choice="auto"
        )
        choice = resp.choices[0]
        
        tool_calls = choice.message.tool_calls
        final_text = None

        if tool_calls:
            # Model requested a tool
            call = tool_calls[0]
            name = call.function.name
            args = json.loads(call.function.arguments)

            result = IMPL[name](**args)

            # Add assistant tool call message
            msgs.append({
                "role": "assistant",
                "tool_calls": [
                    {
                        "id": call.id,
                        "type": "function",
                        "function": {
                            "name": name,
                            "arguments": call.function.arguments
                        }
                    }
                ]
            })

            # Add the tool's result

            msgs.append({
                "role": "tool",
                "tool_call_id": call.id,
                "content": "When you need to use a tool, respond ONLY in JSON: {\"tool\": \"name\", \"args\": {...}}. Otherwise, answer normally."
            })

            # Second call: model produces final answer
            final = client.chat.complete(
                model="mistral-small-latest",
                messages=msgs
            )
            final_text = final.choices[0].message.content
        else:
            # No tool, answer directly
            final_text = choice.message.content

        print(final_text)
        msgs.append({"role": "assistant", "content": final_text})

if __name__ == "__main__":
    run()

Sam Altman is a prominent American entrepreneur, investor, and programmer. He is best known as the former president of Y Combinator, a highly influential startup accelerator. Altman co-founded Loopt, a location-based social networking service, and has been involved in various other ventures and investments in the technology and startup ecosystems. He is also known for his work in artificial intelligence and his involvement with OpenAI.
